# Semantic Kernel with OpenBnB MCP Server Integration

ਇਹ ਨੋਟਬੁੱਕ dikhāuṁdā hai kimeṁ Semantic Kernel nūṁ asal OpenBnB MCP ਸੇਰਵਰ nāl MCPStdioPlugin dīvārā asli Airbnb rahāiś dī talāś karan lai vartiya jā sakadā hai। LLM Access lai, iha Microsoft Foundry dā upayōg kardā hai। Apṇē pāriyāvaraṇ variables set up karan lai, tusīṁ [Setup Lesson ](/00-course-setup/README.md) dē kōrī asan tarīqē dā pālan kar sakadē hō।


## ਲੋੜੀਂਦੇ ਪੈਕੇਜ ਆਯਾਤ ਕਰੋ


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCP ਪਲੱਗਇਨ ਕਨੈਕਸ਼ਨ ਬਣਾਉਣਾ

ਅਸੀਂ MCPStdioPlugin ਦੀ ਵਰਤੋਂ ਕਰਕੇ [OpenBnB MCP ਸਰਵਰ](https://github.com/openbnb-org/mcp-server-airbnb) ਨਾਲ ਜੁੜਾਂਗੇ। ਇਹ ਸਰਵਰ @openbnb/mcp-server-airbnb ਪੈਕੇਜ ਰਾਹੀਂ Airbnb ਖੋਜ ਕਾਰਜਪ੍ਰਣਾਲੀ ਪ੍ਰਦਾਨ ਕਰਦਾ ਹੈ।


## Creating the Client

ਇਸ ਨਮੂਨੇ ਵਿੱਚ, ਅਸੀਂ ਆਪਣੇ LLM ਐਕਸੈੱਸ ਲਈ Microsoft Foundry ਦੀ ਵਰਤੋਂ ਕਰਾਂਗੇ। ਯਕੀਨੀ ਬਣਾਓ ਕਿ ਤੁਹਾਡੇ ਵਾਤਾਵਰਣ ਚਲਨਸ਼ੀਲ ਹਨ।


## Environment Configuration

Azure OpenAI ਸੈਟਿੰਗਸ ਸੰਰਚਿਤ ਕਰੋ। ਯਕੀਨੀ ਬਣਾਓ ਕਿ ਤੁਹਾਡੇ ਕੋਲ ਹੇਠਾਂ ਦਿੱਤੇ ਵਾਤਾਵਰਣ ਚਲ ਵੈਰੀਏਬਲ ਸੈਟ ਹਨ:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP ਇੰਟੀਗ੍ਰੇਸ਼ਨ ਨੂੰ ਸਮਝਣਾ

ਇਹ ਨੋਟਬੁੱਕ **ਅਸਲ OpenBnB MCP ਸਰਵਰ** ਨਾਲ ਜੁੜਦਾ ਹੈ ਜੋ ਅਸਲੀ Airbnb ਖੋਜ ਫੰਕਸ਼ਨਾਲਿਟੀ ਪ੍ਰਦਾਨ ਕਰਦਾ ਹੈ।

### ਇਹ ਕਿਵੇਂ ਕੰਮ ਕਰਦਾ ਹੈ:

1. **MCPStdioPlugin**: MCP ਸਰਵਰ ਨਾਲ ਸਟੈਂਡਰਡ ਇਨਪੁੱਟ/ਆਉਟਪੁੱਟ ਸੰਚਾਰ ਵਰਤਦਾ ਹੈ  
2. **ਅਸਲੀ NPM ਪੈਕੇਜ**: npx ਰਾਹੀਂ `@openbnb/mcp-server-airbnb` ਡਾਊਨਲੋਡ ਅਤੇ ਚਲਾਉਂਦਾ ਹੈ  
3. **ਲਾਈਵ ਡਾਟਾ**: ਉਹਨਾਂ ਦੀਆਂ APIs ਤੋਂ ਅਸਲ Airbnb ਸੰਪਤੀ ਦਾ ਡਾਟਾ ਵਾਪਸ ਕਰਦਾ ਹੈ  
4. **ਫੰਕਸ਼ਨ ਖੋਜ**: ਏਜੰਟ ਆਪਣੇ ਆਪ MCP ਸਰਵਰ ਤੋਂ ਉਪਲਬਧ ਫੰਕਸ਼ਨ ਖੋਜਦਾ ਹੈ  

### ਉਪਲਬਧ ਫੰਕਸ਼ਨ:

OpenBnB MCP ਸਰਵਰ ਆਮ ਤੌਰ 'ਤੇ ਮੁਹੱਈਆ ਕਰਦਾ ਹੈ:  
- **search_listings** - ਸਥਿਤੀ ਅਤੇ ਮਾਪਦੰਡਾਂ ਦੁਆਰਾ Airbnb ਸੰਪਤੀਆਂ ਦੀ ਖੋਜ ਕਰੋ  
- **get_listing_details** - ਖਾਸ ਸੰਪਤੀਆਂ ਬਾਰੇ ਵਿਸਥਾਰਤ ਜਾਣਕਾਰੀ ਪ੍ਰਾਪਤ ਕਰੋ  
- **check_availability** - ਖਾਸ ਤਰੀਕਾਂ ਲਈ ਉਪਲਬਧਤਾ ਜਾਂਚੋ  
- **get_reviews** - ਸੰਪਤੀਆਂ ਲਈ ਸਮੀਖਿਆਵਾਂ ਪ੍ਰਾਪਤ ਕਰੋ  
- **get_host_info** - ਸੰਪਤੀ ਮਾਲਕਾਂ ਬਾਰੇ ਜਾਣਕਾਰੀ ਪ੍ਰਾਪਤ ਕਰੋ  

### ਜਰੂਰੀ ਸ਼ਰਤਾਂ:

- ਤੁਹਾਡੇ ਸਿਸਟਮ ਤੇ **Node.js** ਇੰਸਟਾਲ ਹੋਣਾ ਚਾਹੀਦਾ ਹੈ  
- MCP ਸਰਵਰ ਪੈਕੇਜ ਡਾਊਨਲੋਡ ਕਰਨ ਲਈ **ਇੰਟਰਨੈੱਟ ਕਨੈਕਸ਼ਨ** ਹੋਣਾ ਚਾਹੀਦਾ ਹੈ  
- **NPX** ਉਪਲਬਧ ਹੋਣਾ ਚਾਹੀਦਾ ਹੈ (Node.js ਨਾਲ ਆਉਂਦਾ ਹੈ)  

### ਕਨੈਕਸ਼ਨ ਦੀ ਜਾਂਚ ਕਰਨਾ:

ਤੁਸੀਂ MCP ਸਰਵਰ ਨੂੰ ਹੱਥੋਂ ਟੈਸਟ ਕਰ ਸਕਦੇ ਹੋ:  
```bash
npx -y @openbnb/mcp-server-airbnb
```
  
ਇਹ OpenBnB MCP ਸਰਵਰ ਡਾਊਨਲੋਡ ਅਤੇ ਸ਼ੁਰੂ ਕਰੇਗਾ, ਜਿਸ ਨਾਲ Semantic Kernel ਅਸਲ Airbnb ਡਾਟਾ ਲਈ ਜੁੜਦਾ ਹੈ।


## OpenBnB MCP ਸਰਵਰ ਨਾਲ ਏਜੰਟ ਨੂੰ ਚਲਾਉਣਾ

ਹੁਣ ਅਸੀਂ AI ਏਜੰਟ ਨੂੰ ਚਲਾਵਾਂਗੇ ਜੋ OpenBnB MCP ਸਰਵਰ ਨਾਲ ਜੁੜਦਾ ਹੈ ਤਾਂ ਜੋ ਸਟੌਕਹੋਮ ਵਿੱਚ 2 ਬਾਲਗਾਂ ਅਤੇ 1 ਬੱਚੇ ਲਈ ਅਸਲੀ Airbnb ਰਹਿਣ ਦੀ ਥਾਂਵਾਂ ਦੀ ਖੋਜ ਕਰ ਸਕੇ। ਲੱਭਾਈ ਮਾਪਦੰਡ ਬਦਲਣ ਲਈ `user_inputs` ਸੂਚੀ ਨੂੰ ਬਦਲਣ ਵਿੱਚ ਸਵਤੰਤਰ ਮਹਿਸੂਸ ਕਰੋ।


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Summary
ਵਧਾਈਆਂ! ਤੁਸੀਂ ਇੱਕ ਐਸਾ ਏਆਈ ਏਜੰਟ ਸਫਲਤਾਪੂਰਵਕ ਬਣਾਇਆ ਹੈ ਜੋ ਮਾਡਲ ਕੰਟੈਕਸਟ ਪ੍ਰੋਟੋਕਾਲ (MCP) ਦੇ ਨਾਲ ਅਸਲੀ ਦੁਨੀਆ ਦੀ ਆਵਾਸ ਖੋਜ ਨਾਲ ਇੰਟੀਗ੍ਰੇਟ ਕਰਦਾ ਹੈ:

## Technologies Used:
- ਸੈਮਾਂਟਿਕ ਕਰਨਲ - Azure OpenAI ਨਾਲ ਬੁੱਧੀਮਾਨ ਏਜੰਟ ਬਣਾਉਣ ਲਈ
- ਮਾਈਕ੍ਰੋਸਾਫਟ ਫਾਉਂਡਰੀ - LLM ਸਮਰੱਥਾਵਾਂ ਅਤੇ ਚੈਟ ਸੰਪੂਰਨਤਾ ਲਈ
- MCP (ਮਾਡਲ ਕੰਟੈਕਸਟ ਪ੍ਰੋਟੋਕਾਲ) - ਮਿਆਰੀਕ੍ਰਿਤ ਸੰਦ ਇੰਟੀਗ੍ਰੇਸ਼ਨ ਲਈ
- OpenBnB MCP ਸਰਵਰ - ਅਸਲੀ Airbnb ਖੋਜ ਫੰਕਸ਼ਨਲਿਟੀ ਲਈ
- Node.js/NPX - ਬਾਹਰੀ MCP ਸਰਵਰ ਚਲਾਉਣ ਲਈ

## What You've Learned:
- MCP ਇੰਟੀਗ੍ਰੇਸ਼ਨ: ਸੈਮਾਂਟਿਕ ਕਰਨਲ ਏਜੰਟਾਂ ਨੂੰ ਬਾਹਰੀ MCP ਸਰਵਰਾਂ ਨਾਲ ਜੋੜਨਾ
- ਰੀਅਲ-ਟਾਈਮ ਡਾਟਾ ਐਕਸੇਸ: ਜੀਵੰਤ APIs ਰਾਹੀਂ ਅਸਲ Airbnb ਸੰਪਤੀਆਂ ਦੀ ਖੋਜ
- ਪ੍ਰੋਟੋਕਾਲ ਕਮਿਊਨੀਕੇਸ਼ਨ: ਏਜੰਟ ਅਤੇ MCP ਸਰਵਰ ਦਰਮਿਆਨ stdio ਕਮਿਊਨੀਕੇਸ਼ਨ ਵਰਤਣਾ
- ਫੰਕਸ਼ਨ ਡਿਸਕਵਰੀ: MCP ਸਰਵਰਾਂ ਤੋਂ ਉਪਲਬਧ ਫੰਕਸ਼ਨਾਂ ਨੂੰ ਆਪਣੇ ਆਪ ਖੋਜਣਾ
- ਸਟ੍ਰੀਮਿੰਗ ਰਿਸਪਾਂਸ: ਰੀਅਲ-ਟਾਈਮ ਵਿੱਚ ਫੰਕਸ਼ਨ ਕਾਲਾਂ ਨੂੰ ਕੈਪਚਰ ਅਤੇ ਲੌਗ ਕਰਨਾ
- HTML ਰੇਂਡਰਿੰਗ: ਸਟਾਈਲ ਕੀਤੀਆਂ ਟੇਬਲਾਂ ਅਤੇ ਇੰਟਰਐਕਟਿਵ ਡਿਸਪਲੇਜ਼ ਨਾਲ ਏਜੰਟ ਦੇ ਜਵਾਬਾਂ ਨੂੰ ਫਾਰਮੈਟ ਕਰਨਾ

## Next Steps:
- ਵਾਧੂ MCP ਸਰਵਰਾਂ ਨੂੰ ਜੋੜਨਾ (ਮੌਸਮ, ਉਡਾਣਾਂ, ਰੈਸਟੋਰੈਂਟ)
- MCP ਅਤੇ A2A ਪ੍ਰੋਟੋਕਾਲਜ਼ ਨੂੰ ਮਿਲਾ ਕੇ ਇੱਕ ਬਹੁ-ਏਜੰਟ ਸਿਸਟਮ ਬਣਾਉਣਾ
- ਆਪਣੀਆਂ ਡਾਟਾ ਸਰੋਤਾਂ ਲਈ ਕਸਟਮ MCP ਸਰਵਰ ਬਣਾਉਣਾ
- ਸੈਸ਼ਨਾਂ ਦਰਮਿਆਨ ਸਥਾਈ ਗੱਲਬਾਤ ਯਾਦاشت ਲਾਗੂ ਕਰਨਾ
- MCP ਸਰਵਰ ਆਰਕੀਸਟ੍ਰੇਸ਼ਨ ਨਾਲ ਏਜੰਟ ਨੂੰ Azure Functions 'ਤੇ ਤੈਨਾਤ ਕਰਨਾ
- ਯੂਜ਼ਰ ਪ੍ਰਮਾਣੀਕਰਨ ਅਤੇ ਬੁਕਿੰਗ ਸਮਰੱਥਾਵਾਂ ਸ਼ਾਮਿਲ ਕਰਨਾ


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**ਅਸਪਸ਼ਟੀਕਰਨ**:  
ਇਹ ਦਸਤਾਵੇਜ਼ AI ਅਨੁਵਾਦ ਸੇਵਾ [Co-op Translator](https://github.com/Azure/co-op-translator) ਦੀ ਵਰਤੋਂ ਕਰਕੇ ਅਨੁਵਾਦਿਤ ਕੀਤਾ ਗਿਆ ਹੈ। ਜੇਕਰ ਅਸੀਂ ਸਹੀਤਾ ਲਈ ਯਤਨ ਕਰਦੇ ਹਾਂ, ਤਾਂ ਕਿਰਪਾ ਕਰਕੇ ਧਿਆਨ ਵਿੱਚ ਰੱਖੋ ਕਿ ਆਟੋਮੈਟেড ਅਨੁਵਾਦਾਂ ਵਿੱਚ ਗਲਤੀਆਂ ਜਾਂ ਅਣਸਹੀਤਾਂ ਹੋ ਸਕਦੀਆਂ ਹਨ। ਮੂਲ ਦਸਤਾਵੇਜ਼ ਆਪਣੀ ਮੂਲ ਭਾਸ਼ਾ ਵਿੱਚ ਅਧਿਕਾਰ ਪ੍ਰਮਾਣਿਕ ਸ્રੋਤ ਮੰਨਿਆ ਜਾਣਾ ਚਾਹੀਦਾ ਹੈ। ਮਹੱਤਵਪੂਰਨ ਜਾਣਕਾਰੀ ਲਈ, ਪੇਸ਼ੇਵਰ ਮਨੁੱਖੀ ਅਨੁਵਾਦ ਦੀ ਸਿਫ਼ਾਰਸ਼ ਕੀਤੀ ਜਾਂਦੀ ਹੈ। ਇਸ ਅਨੁਵਾਦ ਦੀ ਵਰਤੋਂ ਨਾਲ ਪੈਦਾਅ ਹੋਣ ਵਾਲੀਆਂ ਕੋਈ ਭੀ ਗਲਤ ਫਹਿਮੀਆਂ ਜਾਂ ਦੁੱਧਾਂਦੇ ਲਈ ਅਸੀਂ ਜ਼ਿੰਮੇਵਾਰ ਨਹੀਂ ਹਾਂ।
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
